In [2]:
from tqdm import tqdm
import os

In [3]:
apikey = ''

In [4]:
import requests

def curl(value):
    headers = {
        'Content-Type': 'application/json',
        'api-key': apikey,
    }

    params = {
        'api-version': '2023-03-15-preview',
    }

    json_data = {
        'messages': [
            {'role': 'system', 'content':'answer only, no need to explain'},
            {
                'role': 'user',
                'content': value,
            },
        ],
        'max_tokens': 100,
        'temperature': 0,
    }

    response = requests.post(
        'https://nous.openai.azure.com/openai/deployments/nous/chat/completions',
        params=params,
        headers=headers,
        json=json_data,
        timeout = 120,
    )
    
    return response

In [47]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [51]:
questions

[{'no': '1',
  'objektif': 'Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.',
  'soalan': 'Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.\nA. hartawan\nB. dermawan\nC. bangsawan\nD. sukarelawan',
  'jawapan': 'D'},
 {'no': '2',
  'objektif': 'Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.',
  'soalan': 'Kebanyakan barang yang disimpan di dalam stor itu telah rosak ____ tikus.\nA. digerit\nB. digigit\nC. dikesip\nD. diketip',
  'jawapan': 'A'},
 {'no': '3',
  'objektif': 'Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.',
  'soalan': 'Shafiq cuba mengubah kedudukan meja kayu itu ___ ruang tamu ___ ruang dapur rumahnya.\nA. dari ... ke\nB. dari ... kepada\nC. daripada ... ke\nD. daripada ... kepada',
  'jawapan': 'A'},
 {'no': '4',
  'objektif': 'Ayat-ayat di bawah mungkin betul dan mungkin salah. Jika terdapat kesalahan, kesalahan itu telah ditandakan d

In [52]:
row = questions[0]
prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
"""
print(prompt)


objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.
soalan: Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.
A. hartawan
B. dermawan
C. bangsawan
D. sukarelawan



In [54]:
r = curl(prompt)

In [59]:
r.json()['choices'][0]['message']['content']

'D. sukarelawan'

In [65]:
for i in tqdm(range(len(questions))):
    row = questions[i]
    prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
    """.strip()
    try:
        r = curl(prompt)
        answer = r.json()['choices'][0]['message']['content']
        questions[i]['output'] = answer
    except:
        pass

100%|███████████████████████████████████████████| 54/54 [01:31<00:00,  1.69s/it]


In [68]:
import json

with open('output-gpt-35-turbo-0613-BM-A-PT3.json', 'w') as fopen:
    json.dump(questions, fopen)

In [66]:
filtered = [q for q in questions if 'output' in q]
len(filtered)

53

In [76]:
q['jawapan']

'B'

In [77]:
correct = 0
for q in filtered:
    correct += q['output'].split('.')[0] == q['jawapan']

In [80]:
(correct / len(filtered)) * 100

37.735849056603776